# Build a model to calculate the probability that a car is made by one of the  
# top 25 manufacturers in this dataset based on the uncorrupted features

In [6]:
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
#import packages for hyperparameters tuning
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split

In [7]:
data=pd.read_csv(r'C:\Users\hp\Documents\Assesment_2\parking_citations.corrupted.csv',parse_dates=['Issue Date'])

C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
data.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21,1251.0,NaN,NaN,CA,200304.0,NaN,NaN,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
1,1103700150,2015-12-21,1435.0,NaN,NaN,CA,201512.0,NaN,NaN,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
2,1104803000,2015-12-21,2055.0,NaN,NaN,CA,201503.0,NaN,NaN,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4
3,1104820732,2015-12-26,1515.0,NaN,NaN,CA,NaN,NaN,NaN,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2
4,1105461453,2015-09-15,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0


In [9]:
data.shape

(8726014, 19)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8726014 entries, 0 to 8726013
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Ticket number          object        
 1   Issue Date             datetime64[ns]
 2   Issue time             float64       
 3   Meter Id               object        
 4   Marked Time            float64       
 5   RP State Plate         object        
 6   Plate Expiry Date      float64       
 7   VIN                    object        
 8   Make                   object        
 9   Body Style             object        
 10  Color                  object        
 11  Location               object        
 12  Route                  object        
 13  Agency                 float64       
 14  Violation code         object        
 15  Violation Description  object        
 16  Fine amount            float64       
 17  Latitude               float64       
 18  Longitude             

In [11]:
#data_25=data['Make'].value_counts()[:25]

In [12]:
#data_25=pd.DataFrame(data_25)

In [13]:
#data_25.drop(['Make'],axis=1,inplace=True)

In [14]:
#data_25

In [15]:
#data['classification']=data['Make'].replace(to_replace=["TOYT","HOND","FORD","NISS","CHEV","BMW","MERZ","VOLK","HYUN","DODG","LEXS","KIA","JEEP","AUDI","MAZD","OTHR","GMC","CHRY","INFI","ACUR","SUBA","VOLV","TOYO","MITS","CADI"],value=1)

In [16]:
data_train = data[data['Make'].notna()]

In [17]:
data_train

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
4,1105461453,2015-09-15,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,9.999900e+04,9.999900e+04
6,1106500452,2015-12-17,1710.0,NaN,NaN,CA,201605.0,NaN,MAZD,PA,BL,SUNSET/ALVARADO,00217,1.0,8070,PARK IN GRID LOCK ZN,163.0,9.999900e+04,9.999900e+04
10,1106506424,2015-12-22,1100.0,NaN,NaN,CA,201511.0,NaN,FORD,TR,WH,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,9.999900e+04,9.999900e+04
11,1106506435,2015-12-22,1105.0,NaN,NaN,CA,201701.0,NaN,CHRY,PA,GO,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,9.999900e+04,9.999900e+04
14,1107179581,2015-12-27,1055.0,NaN,NaN,CA,201605.0,NaN,TOYO,PA,BK,3100 N HOLLYRIDGE DR,NaN,54.0,8058L,PREF PARKING,68.0,9.999900e+04,9.999900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726006,4347602361,2019-01-10,810.0,VN752,NaN,CA,NaN,NaN,FORD,PA,GY,6301 SYLMAR AV,00300,53.0,88.13B+,METER EXP.,63.0,6.426622e+06,1.889882e+06
8726008,4347602383,2019-01-10,1244.0,VN687,NaN,CA,201909.0,NaN,PORS,PA,BK,14301 DELANO ST,00300,53.0,88.13B+,METER EXP.,63.0,6.427279e+06,1.889149e+06
8726010,4347602405,2019-01-10,1350.0,VN316,NaN,CA,201905.0,NaN,HYUN,PA,SL,14500 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.425963e+06,1.890254e+06
8726011,4347602416,2019-01-10,1354.0,VN914,NaN,CA,201912.0,NaN,NISS,PA,GY,14400 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.426623e+06,1.890252e+06


In [18]:
data_test=data[data['Make'].isna()]

In [19]:
data_train.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
4,1105461453,2015-09-15,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0
6,1106500452,2015-12-17,1710.0,NaN,NaN,CA,201605.0,NaN,MAZD,PA,BL,SUNSET/ALVARADO,00217,1.0,8070,PARK IN GRID LOCK ZN,163.0,99999.0,99999.0
10,1106506424,2015-12-22,1100.0,NaN,NaN,CA,201511.0,NaN,FORD,TR,WH,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,99999.0,99999.0
11,1106506435,2015-12-22,1105.0,NaN,NaN,CA,201701.0,NaN,CHRY,PA,GO,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,99999.0,99999.0
14,1107179581,2015-12-27,1055.0,NaN,NaN,CA,201605.0,NaN,TOYO,PA,BK,3100 N HOLLYRIDGE DR,NaN,54.0,8058L,PREF PARKING,68.0,99999.0,99999.0


In [20]:
data_25=data_train['Make'].value_counts()[:25]

In [21]:
data_25=pd.DataFrame(data_25)

In [22]:
data_25

,Make
TOYT,721411
HOND,491961
FORD,382695
NISS,311324
CHEV,297076
BMW,199221
MERZ,177307
VOLK,149501
HYUN,133864
DODG,127764


In [23]:
data_train['classification']=data_train['Make'].replace(to_replace=["TOYT","HOND","FORD","NISS","CHEV","BMW","MERZ","VOLK","HYUN","DODG","LEXS","KIA","JEEP","AUDI","MAZD","OTHR","GMC","CHRY","INFI","ACUR","SUBA","VOLV","TOYO","MITS","CADI"],value='correct')

<ipython-input-23-2935b1819ea6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['classification']=data_train['Make'].replace(to_replace=["TOYT","HOND","FORD","NISS","CHEV","BMW","MERZ","VOLK","HYUN","DODG","LEXS","KIA","JEEP","AUDI","MAZD","OTHR","GMC","CHRY","INFI","ACUR","SUBA","VOLV","TOYO","MITS","CADI"],value='correct')


In [24]:
data_train['classification']

4          correct
6          correct
10         correct
11         correct
14         correct
            ...   
8726006    correct
8726008       PORS
8726010    correct
8726011    correct
8726013    correct
Name: classification, Length: 4357544, dtype: object

In [25]:
data_train.loc[data_train['classification'] !='correct', 'classification'] = 'wrong'

C:\Users\hp\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [26]:
data_train['classification'].value_counts()

correct    3986214
wrong       371330
Name: classification, dtype: int64

In [27]:
data_train['classification']=data_train['classification'].replace({'correct': 1, 'wrong': 0})

<ipython-input-27-8954ec2533d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['classification']=data_train['classification'].replace({'correct': 1, 'wrong': 0})


In [28]:
data_train.shape

(4357544, 20)

In [29]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4357544 entries, 4 to 8726013
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Ticket number          object        
 1   Issue Date             datetime64[ns]
 2   Issue time             float64       
 3   Meter Id               object        
 4   Marked Time            float64       
 5   RP State Plate         object        
 6   Plate Expiry Date      float64       
 7   VIN                    object        
 8   Make                   object        
 9   Body Style             object        
 10  Color                  object        
 11  Location               object        
 12  Route                  object        
 13  Agency                 float64       
 14  Violation code         object        
 15  Violation Description  object        
 16  Fine amount            float64       
 17  Latitude               float64       
 18  Longitude             

In [30]:
X=data_train.drop(['classification','Ticket number',"Meter Id","RP State Plate","VIN","Make","Body Style","Color","Location","Route","Violation code","Violation Description","Issue Date","Latitude","Longitude"],axis=1)

In [31]:
X.head()

,Issue time,Marked Time,Plate Expiry Date,Agency,Fine amount
4,115.0,NaN,200316.0,1.0,93.0
6,1710.0,NaN,201605.0,1.0,163.0
10,1100.0,NaN,201511.0,1.0,93.0
11,1105.0,NaN,201701.0,1.0,93.0
14,1055.0,NaN,201605.0,54.0,68.0


In [32]:
Y=data_train['classification'].values

In [33]:
Y.shape

(4357544,)

In [34]:
X.fillna(X.mean(),inplace=True)

In [35]:
X.head()

,Issue time,Marked Time,Plate Expiry Date,Agency,Fine amount
4,115.0,1055.147196,200316.0,1.0,93.0
6,1710.0,1055.147196,201605.0,1.0,163.0
10,1100.0,1055.147196,201511.0,1.0,93.0
11,1105.0,1055.147196,201701.0,1.0,93.0
14,1055.0,1055.147196,201605.0,54.0,68.0


In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.50,stratify=Y)

In [37]:
X_train.shape,X_test.shape,y_test.shape,y_train.shape

((2178772, 5), (2178772, 5), (2178772,), (2178772,))

In [38]:
clf=XGBClassifier()

In [39]:
classifier=clf.fit(X_train,y_train)

C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:10:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [40]:
y_pred=classifier.predict(X_test)
y_pred_probabality = classifier.predict_proba(X_test)

In [41]:
y_pred_probabality.max()

0.9947702

In [42]:
y_pred_probabality

array([[0.06426185, 0.93573815],
       [0.08094323, 0.9190568 ],
       [0.05351377, 0.94648623],
       ...,
       [0.09275496, 0.90724504],
       [0.07328272, 0.9267173 ],
       [0.04686379, 0.9531362 ]], dtype=float32)

In [43]:
probab_df = pd.DataFrame(y_pred_probabality)

In [44]:
probab_df

,0,1
0,0.064262,0.935738
1,0.080943,0.919057
2,0.053514,0.946486
3,0.065980,0.934020
4,0.053370,0.946630
...,...,...
2178767,0.064412,0.935588
2178768,0.068760,0.931240
2178769,0.092755,0.907245
2178770,0.073283,0.926717


In [45]:
from sklearn.metrics import confusion_matrix,classification_report
cm_df=pd.DataFrame(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.02      0.66      0.04      5287
           1       1.00      0.92      0.96   2173485

    accuracy                           0.92   2178772
   macro avg       0.51      0.79      0.50   2178772
weighted avg       1.00      0.92      0.95   2178772



In [46]:
score = accuracy_score(y_test,y_pred)
score

0.9155753791585352

# Saving the model to pickle file 

In [47]:
import pickle
pickle_file=open("classifier_parking_citations_corrupted.pkl","wb")
pickle.dump(classifier,pickle_file)
pickle_file.close()

In [48]:
X.columns

Index(['Issue time', 'Marked Time', 'Plate Expiry Date', 'Agency',
       'Fine amount'],
      dtype='object')

In [53]:
X.head

<bound method NDFrame.head of          Issue time  Marked Time  Plate Expiry Date  Agency  Fine amount
4             115.0  1055.147196      200316.000000     1.0         93.0
6            1710.0  1055.147196      201605.000000     1.0        163.0
10           1100.0  1055.147196      201511.000000     1.0         93.0
11           1105.0  1055.147196      201701.000000     1.0         93.0
14           1055.0  1055.147196      201605.000000    54.0         68.0
...             ...          ...                ...     ...          ...
8726006       810.0  1055.147196      186782.570357    53.0         63.0
8726008      1244.0  1055.147196      201909.000000    53.0         63.0
8726010      1350.0  1055.147196      201905.000000    53.0         63.0
8726011      1354.0  1055.147196      201912.000000    53.0         63.0
8726013      1414.0  1055.147196      201905.000000    53.0         63.0

[4357544 rows x 5 columns]>